# Compare DMS to natural sequence evolution

In [1]:
# this cell is tagged parameters for papermill parameterization
dms_summary_csv = None
growth_rates_csv = None
pango_consensus_seqs_json = None
starting_clades = None
dms_clade = None
n_random = None
exclude_clades = None
pango_dms_phenotypes_csv = None
pango_by_date_html = None
pango_affinity_vs_escape_html = None
pango_dms_vs_growth_regression_html = None
pango_dms_vs_growth_regression_by_domain_html = None
pango_dms_vs_growth_corr_html = None
pango_dms_vs_growth_corr_by_domain_html = None
exclude_clades_with_muts = None

In [2]:
# Parameters
starting_clades = ["BA.2", "BA.5", "XBB"]
starting_clades = ["XBB"]
dms_clade = "XBB.1.5"
dms_summary_csv = "results/summaries/summary.csv"
growth_rates_csv = "MultinomialLogisticGrowth/model_fits/rates.csv"
pango_consensus_seqs_json = (
    "results/compare_natural/pango-consensus-sequences_summary.json"
)
pango_dms_phenotypes_csv = "results/compare_natural/pango_dms_phenotypes.csv"
pango_by_date_html = "results/compare_natural/pango_dms_phenotypes_by_date.html"
pango_affinity_vs_escape_html = "results/compare_natural/pango_affinity_vs_escape.html"
pango_dms_vs_growth_regression_html = (
    "results/compare_natural/pango_dms_vs_growth_regression.html"
)
pango_dms_vs_growth_regression_by_domain_html = (
    "results/compare_natural/pango_dms_vs_growth_regression_by_domain.html"
)
pango_dms_vs_growth_corr_html = "results/compare_natural/pango_dms_vs_growth_corr.html"
pango_dms_vs_growth_corr_by_domain_html = (
    "results/compare_natural/pango_dms_vs_growth_corr_by_domain.html"
)
n_random = 10
exclude_clades = []
exclude_clades_with_muts = []

import os
os.chdir("../")


In [3]:
import collections
import itertools
import json
import math
import re

import altair as alt

import numpy

import pandas as pd

import polyclonal.plot

import scipy.stats

import statsmodels.api

_ = alt.data_transformers.disable_max_rows()

## Read Pango clades and mutations

In [4]:
with open(pango_consensus_seqs_json) as f:
    pango_clades = json.load(f)

def n_child_clades(c):
    """Get number of children clades of a Pango clade."""
    direct_children = pango_clades[c]["children"]
    return len(direct_children) + sum([n_child_clades(c_child) for c_child in direct_children])

def build_records(c, recs):
    """Build records of Pango clade information."""
    if c in recs["clade"]:
        return
    recs["clade"].append(c)
    recs["n_child_clades"].append(n_child_clades(c))
    recs["date"].append(pango_clades[c]["designationDate"])
    recs["muts_from_ref"].append(
        [
            mut.split(":")[1]
            for field in ["aaSubstitutions", "aaDeletions"]
            for mut in pango_clades[c][field]
            if mut and mut.startswith("S:")
        ]
    )
    recs["parent"].append(pango_clades[c]["parent"])
    recs["all_new_muts_from_ref"].append(
        [
            mut
            for field in ["aaSubstitutionsNew", "aaDeletionsNew"]
            for mut in pango_clades[c][field]
            if mut
        ]
    )
    recs["all_new_muts_reverted_from_ref"].append(
        [
            mut
            for field in ["aaSubstitutionsReverted", "aaDeletionsReverted"]
            for mut in pango_clades[c][field]
            if mut
        ]
    )
    for c_child in pango_clades[c]["children"]:
        build_records(c_child, recs)
        
records = collections.defaultdict(list)
for starting_clade in starting_clades:
    build_records(starting_clade, records)

pango_df = pd.DataFrame(records).query("clade not in @exclude_clades")
dms_clade_mutations_from_ref = pango_df.set_index("clade").at[
    dms_clade, "muts_from_ref"
]

def mutations_from(muts, from_muts):
    """Get mutations from another sequence."""
    new_muts = set(muts).symmetric_difference(from_muts)
    assert all(re.fullmatch("[A-Z\-]\d+[A-Z\-]", m) for m in new_muts)
    new_muts_d = collections.defaultdict(list)
    for m in new_muts:
        new_muts_d[int(m[1: -1])].append(m)
    new_muts_list = []
    for _, ms in sorted(new_muts_d.items()):
        if len(ms) == 1:
            m = ms[0]
            if m in muts:
                new_muts_list.append(m)
            else:
                assert m in from_muts
                new_muts_list.append(m[-1] + m[1: -1] + m[0])
        else:
            m, from_m = ms
            if m not in muts:
                from_m, m = m, from_m
            assert m in muts and from_m in from_muts
            new_muts_list.append(from_m[-1] + m[1: ])
    return new_muts_list

pango_df = (
    pango_df
    .assign(
        muts_from_dms_clade=lambda x: x["muts_from_ref"].apply(
            mutations_from, args=(dms_clade_mutations_from_ref,),
        ),
        date=lambda x: pd.to_datetime(x["date"]),
    )
    .drop(columns="muts_from_ref")
    .sort_values("date")
    .reset_index(drop=True)
)

for mut in exclude_clades_with_muts:
    pango_df = pango_df[pango_df["muts_from_dms_clade"].map(lambda ms: mut not in ms)]

pango_df

,clade,n_child_clades,date,parent,all_new_muts_from_ref,all_new_muts_reverted_from_ref,muts_from_dms_clade
0,XBB,657,2022-09-17,,"[E:T9I, E:T11A, M:Q19E, M:A63T, N:P13L, N:R203...",[],"[V252G, P486S]"
1,XBB.1,568,2022-10-03,XBB,[S:G252V],[],[P486S]
2,XBB.1.1,0,2022-10-15,XBB.1,"[ORF1a:P309L, ORF8:G8*]",[],[P486S]
3,XBB.2,71,2022-10-15,XBB,[S:D253G],[],"[V252G, D253G, P486S]"
4,XBB.3,5,2022-10-15,XBB,"[ORF1a:G82D, ORF1b:I1998V]",[],"[V252G, P486S]"
...,...,...,...,...,...,...,...
653,GJ.1.2.7,0,2023-10-04,GJ.1.2,"[ORF1a:H919Y, S:T478R]",[S:T478K],"[K182N, V252G, D253G, K478R, P521S]"
654,GJ.1.2.8,0,2023-10-04,GJ.1.2,"[ORF3a:G100C, S:T747I]",[],"[K182N, V252G, D253G, P521S, T747I]"
655,GJ.5,1,2023-10-04,XBB.2.3.3,[S:T478R],[S:T478K],"[K182N, V252G, D253G, K478R, P521S]"
656,GJ.5.1,0,2023-10-04,GJ.5,"[ORF1a:A540V, ORF1a:A3963V, S:S255F]",[],"[K182N, V252G, D253G, S255F, K478R, P521S]"


## Assign DMS phenotypes to Pango clades

First define function that assigns DMS phenotypes to mutations:

In [5]:
# read the DMS data
dms_summary = pd.read_csv(dms_summary_csv).rename(
    columns={
        "spike mediated entry": "cell entry",
        "human sera escape": "sera escape",
    }
)

# specify DMS phenotypes of interest
phenotypes = [
    "sera escape",
    "ACE2 affinity",
    "cell entry",
]
assert set(phenotypes).issubset(dms_summary.columns)

phenotype_colors = {
    "sera escape": "red",
    "ACE2 affinity": "blue",
    "cell entry": "purple",
}
assert set(phenotypes) == set(phenotype_colors)


# dict that maps site to wildtype in DMS
dms_wt = dms_summary.set_index("site")["wildtype"].to_dict()

# dict that maps site to region in DMS
site_to_region = dms_summary.set_index("site")["region"].to_dict()

def mut_dms(m, dms_data):
    """Get DMS phenotypes for a mutation."""
    null_d = {k: pd.NA for k in phenotypes}
    if pd.isnull(m) or int(m[1: -1]) not in dms_wt:
        d = null_d
        d["is_RBD"] = pd.NA
    else:
        parent = m[0]
        site = int(m[1: -1])
        mut = m[-1]
        wt = dms_wt[site]
        if parent == wt:
            try:
                d = dms_data[(site, parent, mut)]
            except KeyError:
                d = null_d
        elif mut == wt:
            try:
                d = {k: -v for (k, v) in dms_data[(site, mut, parent)].items()}
            except KeyError:
                d = null_d
        else:
            try:
                parent_d = dms_data[(site, wt, parent)]
                mut_d = dms_data[(site, wt, mut)]
                d = {p: mut_d[p] - parent_d[p] for p in phenotypes}
            except KeyError:
                d = null_d
        d["is_RBD"] = (site_to_region[site] == "RBD")
    assert list(d) == phenotypes + ["is_RBD"]
    return d

Now assign phenotypes to pango clades.
We do this both using the actual DMS data and randomizing the DMS data among measured mutations:

In [6]:
def get_pango_dms_df(dms_data_dict):
    """Given dict mapping mutations to DMS data, get data frame of values for Pango clades."""
    pango_dms_df = (
        pango_df
        # put one mutation in each column
        .explode("muts_from_dms_clade")
        .rename(columns={"muts_from_dms_clade": "mutation"})
        # to add multiple columns: https://stackoverflow.com/a/46814360
        .apply(
            lambda cols: pd.concat([cols, pd.Series(mut_dms(cols["mutation"], dms_data_dict))]),
            axis=1,
        )
        .melt(
            id_vars=["clade", "date", "n_child_clades", "mutation", "is_RBD"],
            value_vars=phenotypes,
            var_name="DMS_phenotype",
            value_name="mutation_effect",
        )
        .assign(
            muts_from_dms_clade=lambda x: x.groupby(["clade", "DMS_phenotype"])["mutation"].transform(
                lambda ms: "; ".join([m for m in ms if not pd.isnull(m)])
            ),
            mutation_missing=lambda x: x["mutation"].where(
                x["mutation_effect"].isnull() & x["mutation"].notnull(),
                pd.NA,
            ),
            muts_from_dms_clade_missing_data=lambda x: (
                x.groupby(["clade", "DMS_phenotype"])["mutation_missing"]
                .transform(lambda ms: "; ".join([m for m in ms if not pd.isnull(m)]))
            ),
            mutation_effect=lambda x: x["mutation_effect"].fillna(0),
            is_RBD=lambda x: x["is_RBD"].fillna(False),
            mutation_effect_RBD=lambda x: x["mutation_effect"] * x["is_RBD"].astype(int),
            mutation_effect_nonRBD=lambda x: x["mutation_effect"] * (~x["is_RBD"]).astype(int),
        )
        .groupby(
            [
                "clade",
                "date",
                "n_child_clades",
                "muts_from_dms_clade",
                "muts_from_dms_clade_missing_data",
                "DMS_phenotype",
            ],
            as_index=False,
        )
        .aggregate(
            phenotype=pd.NamedAgg("mutation_effect", "sum"),
            phenotype_RBD_only=pd.NamedAgg("mutation_effect_RBD", "sum"),
            phenotype_nonRBD_only=pd.NamedAgg("mutation_effect_nonRBD", "sum"),
        )
        .rename(
            columns={
                "muts_from_dms_clade": f"muts_from_{dms_clade}",
                "muts_from_dms_clade_missing_data": f"muts_from_{dms_clade}_missing_data",
            },
        )
        .sort_values(["date", "DMS_phenotype"])
        .reset_index(drop=True)
    )
    
    assert set(pango_df["clade"]) == set(pango_dms_df["clade"])
    assert numpy.allclose(
        pango_dms_df["phenotype"],
        pango_dms_df["phenotype_RBD_only"] + pango_dms_df["phenotype_nonRBD_only"]
    )

    return pango_dms_df

# First, get the actual DMS data mapped to phenotype
dms_data_dict_actual = (
    dms_summary
    .set_index(["site", "wildtype", "mutant"])
    [phenotypes]
    .to_dict(orient="index")
)
pango_dms_df = get_pango_dms_df(dms_data_dict_actual)
print(f"Saving Pango DMS phenotypes to {pango_dms_phenotypes_csv}")
pango_dms_df.to_csv(pango_dms_phenotypes_csv, float_format="%.4f", index=False)

# Now get the randomized DMS data mapped to phenotype
pango_dms_dfs_rand = []
numpy.random.seed(0)
for irandom in range(1, n_random + 1):
    # randomize the non-null DMS data for each phenotype
    dms_summary_rand = dms_summary.copy()
    for phenotype in phenotypes:
        dms_summary_rand = dms_summary_rand.assign(
            **{phenotype: lambda x: numpy.random.permutation(x[phenotype].values)}
        )
    dms_data_dict_rand = (
        dms_summary_rand
        .set_index(["site", "wildtype", "mutant"])
        [phenotypes]
        .to_dict(orient="index")
    )
    pango_dms_dfs_rand.append(get_pango_dms_df(dms_data_dict_rand).assign(randomize=irandom))
# all randomizations concatenated
pango_dms_df_rand = pd.concat(pango_dms_dfs_rand)

Saving Pango DMS phenotypes to results/compare_natural/pango_dms_phenotypes.csv


## Plot phenotypes of Pango clades
Plot phenotypes of Pango clades versus their designation dates:

In [7]:
region_cols = {
    "phenotype": "full spike",
    "phenotype_RBD_only": "RBD only",
    "phenotype_nonRBD_only": "non-RBD only",
}

pango_chart_df = (
    pango_dms_df
    .melt(
        id_vars=[c for c in pango_dms_df if c not in region_cols],
        value_vars=region_cols,
        var_name="spike_region",
        value_name="phenotype value",
    )
    .assign(
        spike_region=lambda x: x["spike_region"].map(region_cols),
    )
    .rename(columns={f"muts_from_{dms_clade}_missing_data": "muts_missing_data"})
)

# columns cannot have "." in them for Altair
col_renames = {c: c.replace(".", "_") for c in pango_chart_df.columns if "." in c}
col_renames_rev = {v: k for (k, v) in col_renames.items()}
pango_chart_df = pango_chart_df.rename(columns=col_renames)

clade_selection = alt.selection_point(fields=["clade"], on="mouseover", empty=False)

base_pango_chart = (
    alt.Chart(pango_chart_df)
    .encode(
        tooltip=[
            alt.Tooltip(c, title=col_renames_rev[c] if c in col_renames_rev else c)
            for c in pango_chart_df.columns
        ],
        opacity=alt.condition(clade_selection, alt.value(1), alt.value(0.35)),
        size=alt.condition(clade_selection, alt.value(60), alt.value(40)),
        strokeWidth=alt.condition(clade_selection, alt.value(2), alt.value(0)),
        color=alt.Color(
            "DMS_phenotype",
            legend=None,
            scale=alt.Scale(
                range=list(phenotype_colors.values()),
                domain=list(phenotype_colors.keys()),
            ),
        ),
    )
    .mark_circle(stroke="black")
    .properties(width=300, height=125)
)

phenotype_pango_charts = []
for phenotype in phenotypes:
    first_row = (phenotype == phenotypes[0])
    last_row = (phenotype == phenotypes[-1])
    phenotype_pango_charts.append(
        base_pango_chart
        .transform_filter(alt.datum["DMS_phenotype"] == phenotype)
        .encode(
            x=alt.X(
                "date",
                title="designation date of clade" if last_row else None,
                axis=(
                    alt.Axis(titleFontSize=12, labelOverlap=True, format="%b-%Y", labelAngle=-90)
                    if last_row
                    else None
                ),
                scale=alt.Scale(nice=False, padding=3),
            ),
            y=alt.Y(
                "phenotype value",
                title=phenotype,
                axis=alt.Axis(titleFontSize=12),
                scale=alt.Scale(nice=False, padding=3),
            ),
            column=alt.Column(
                "spike_region",
                sort=list(region_cols),
                title=None,
                header=(
                    alt.Header(labelFontSize=12, labelFontStyle="bold", labelPadding=4)
                    if first_row
                    else None
                ),
                spacing=4,
            ),
        )
    )

pango_chart = (
    alt.vconcat(*phenotype_pango_charts, spacing=4)
    .configure_axis(grid=False)
    .add_params(clade_selection)
    .properties(        
        title=alt.TitleParams(
            f"DMS predicted phenotypes of Pango clades descended from {', '.join(starting_clades)}",
            anchor="middle",
            fontSize=16,
            dy=-5,
        ),
    )
)

print(f"Saving chart to {pango_by_date_html}")
pango_chart.save(pango_by_date_html)

pango_chart

Saving chart to results/compare_natural/pango_dms_phenotypes_by_date.html


alt.VConcatChart(...)

## Pango clade phenotype correlation

In [8]:
pango_scatter_df = (
    pango_dms_df
    .pivot_table(
        index=[
            c
            for c in pango_dms_df
            if c not in {"DMS_phenotype", "phenotype", "phenotype_RBD_only", "phenotype_nonRBD_only", "n_child_clades"}
        ],
        values="phenotype",
        columns="DMS_phenotype",
    )
    .reset_index()
    .rename(columns={f"muts_from_{dms_clade}_missing_data": "muts_missing_data"})
    .rename(columns=col_renames)
)

pango_scatter_base = (
    alt.Chart(pango_scatter_df)
    .add_params(clade_selection)
    .properties(width=110, height=110)
)

pango_scatter_charts = []
for pheno1, pheno2 in itertools.combinations(phenotypes, 2):
    pheno_pango_scatter_chart = (
        pango_scatter_base
        .encode(
            x=alt.X(
                pheno1,
                scale=alt.Scale(nice=False, padding=5),
            ),
            y=alt.Y(
                pheno2,
                scale=alt.Scale(nice=False, padding=5),
            ),
            tooltip=[
                alt.Tooltip(
                    c,
                    title=col_renames_rev[c] if c in col_renames_rev else c,
                    format=".2f" if pango_scatter_df[c].dtype == float else {},
                )
                for c in pango_scatter_df.columns
            ],
            opacity=alt.condition(clade_selection, alt.value(1), alt.value(0.2)),
            size=alt.condition(clade_selection, alt.value(60), alt.value(30)),
            strokeWidth=alt.condition(clade_selection, alt.value(2), alt.value(0)),
        )
        .mark_circle(stroke="red", color="black")
    )
    pheno_pango_r = (
        pango_scatter_base
        .transform_regression(pheno1, pheno2, params=True)
        .transform_calculate(
            r=alt.expr.if_(
                alt.datum["coef"][1] >= 0,
                alt.expr.sqrt(alt.datum["rSquared"]),
                -alt.expr.sqrt(alt.datum["rSquared"]),
            ),
            label='"r = " + format(datum.r, ".2f")',
        )
        .mark_text(align="left", color="orange", fontWeight=500, fontSize=11, opacity=1)
        .encode(
            x=alt.value(3),
            y=alt.value(105),
            text=alt.Text("label:N"),
        )
    )
    pango_scatter_charts.append(pheno_pango_scatter_chart + pheno_pango_r)

pango_scatter_chart = (
    alt.hconcat(*pango_scatter_charts, spacing=9)
    .configure_axis(grid=False)
    .properties(        
        title=alt.TitleParams(
            (
                "Correlations in DMS phenotypes for"
                + f" Pango clades descended from {', '.join(starting_clades)}"
            ),
            anchor="middle",
            fontSize=12,
            dy=-3,
        ),
    )    
)

pango_scatter_chart

alt.HConcatChart(...)

## All mutation phenotype correlation
Now make plots like in prior section, but for mutations rather than clades.
You can select all mutations or just those found in Pango clades:

In [9]:
# get all mutations in Pango clades that are mutations or reversions
# relative to DMS measured mutations
pango_muts_or_rev = []
for m in set(pango_df.explode("muts_from_dms_clade")["muts_from_dms_clade"]):
    if not pd.isnull(m):
        pango_muts_or_rev.append(m)
        pango_muts_or_rev.append(f"{m[-1]}{m[1: -1]}{m[0]}")

In [10]:
mut_scatter_df = (
    dms_summary
    .assign(mutation=lambda x: x["wildtype"] + x["site"].astype(str) + x["mutant"])
    .drop(columns=["wildtype", "site", "mutant", "sequential_site"])
)

mut_selection = alt.selection_point(fields=["mutation"], on="mouseover", empty=False)

only_muts_in_pango = alt.param(
    value=False,
    bind=alt.binding_radio(
        options=[True, False],
        name=f"only mutations in Pango clades descended from {', '.join(starting_clades)}",
    ),
)

mut_scatter_base = (
    alt.Chart(mut_scatter_df)
    .add_params(mut_selection, only_muts_in_pango)
    .transform_calculate(keep_all=alt.expr.if_(only_muts_in_pango, False, True))
    .transform_filter(
        {
            "or": [
                alt.FieldOneOfPredicate(field="mutation", oneOf=pango_muts_or_rev),
                alt.datum["keep_all"],
            ]
        }
    )
    .properties(width=110, height=110)
)

mut_scatter_charts = []
for pheno1, pheno2 in itertools.combinations(phenotypes, 2):
    pheno_mut_scatter_chart = (
        mut_scatter_base
        .encode(
            x=alt.X(
                pheno1,
                scale=alt.Scale(nice=False, padding=5),
            ),
            y=alt.Y(
                pheno2,
                scale=alt.Scale(nice=False, padding=5),
            ),
            tooltip=[
                alt.Tooltip(
                    c,
                    format=".2f" if mut_scatter_df[c].dtype == float else {},
                )
                for c in mut_scatter_df.columns
            ],
            opacity=alt.condition(mut_selection, alt.value(1), alt.value(0.2)),
            size=alt.condition(mut_selection, alt.value(60), alt.value(30)),
            strokeWidth=alt.condition(mut_selection, alt.value(2), alt.value(0)),
        )
        .mark_circle(stroke="red", color="black")
    )
    pheno_mut_r = (
        mut_scatter_base
        .transform_regression(pheno1, pheno2, params=True)
        .transform_calculate(
            r=alt.expr.if_(
                alt.datum["coef"][1] >= 0,
                alt.expr.sqrt(alt.datum["rSquared"]),
                -alt.expr.sqrt(alt.datum["rSquared"]),
            ),
            label='"r = " + format(datum.r, ".2f")',
        )
        .mark_text(align="left", color="orange", fontWeight=500, fontSize=11, opacity=1)
        .encode(
            x=alt.value(3),
            y=alt.value(105),
            text=alt.Text("label:N"),
        )
    )
    mut_scatter_charts.append(pheno_mut_scatter_chart + pheno_mut_r)

mut_scatter_chart = (
    alt.hconcat(*mut_scatter_charts, spacing=9)
    .configure_axis(grid=False)
    .properties(        
        title=alt.TitleParams(
            "Correlations in DMS phenotypes for mutations",
            anchor="middle",
            fontSize=12,
            dy=-3,
        ),
    )    
)

mut_scatter_chart

alt.HConcatChart(...)

## Correlate with clade growth

In [11]:
growth_rates = pd.read_csv(growth_rates_csv).rename(
    columns={"pango": "clade", "seq_volume": "number sequences"}
)

if (invalid_clades := set(growth_rates["clade"]) - set(pango_clades)):
    raise ValueError(f"Growth rates specified for {invalid_clades}")

pango_dms_growth_df = pango_dms_df.merge(growth_rates, on="clade", validate="many_to_one")

pango_dms_growth_df_rand = pango_dms_df_rand.merge(growth_rates, on="clade", validate="many_to_one")

print(
    f"{growth_rates['clade'].nunique()} clades have growth rates estimates.\n"
    f"{pango_dms_df['clade'].nunique()} clades have DMS estimates.\n"
    f"{pango_dms_growth_df['clade'].nunique()} clades have growth and DMS estimates"
)

print("Simple correlations:")
display(
    pango_dms_growth_df
    .groupby("DMS_phenotype")
    [["R", "phenotype", "phenotype_RBD_only", "phenotype_nonRBD_only"]]
    .corr()
    [["R"]]
)

990 clades have growth rates estimates.
658 clades have DMS estimates.
304 clades have growth and DMS estimates
Simple correlations:


R
DMS_phenotype                                
ACE2 affinity R                      1.000000
              phenotype             -0.089235
              phenotype_RBD_only    -0.170808
              phenotype_nonRBD_only  0.125113
cell entry    R                      1.000000
              phenotype              0.342665
              phenotype_RBD_only     0.432374
              phenotype_nonRBD_only  0.135646
sera escape   R                      1.000000
              phenotype              0.554329
              phenotype_RBD_only     0.710630
              phenotype_nonRBD_only -0.076351

Plot number of sequences versus date, with sizes proportional to log of number of sequences in clade:

In [12]:
(
    alt.Chart(pango_dms_growth_df)
    .encode(
        x="date",
        y="R",
        size=alt.Size("number sequences", scale=alt.Scale(type="log")),
        tooltip=pango_dms_growth_df.columns.tolist(),
    )
    .mark_circle(opacity=0.25, color="black")
)

alt.Chart(...)

Now perform OLS, weighting clades by log number of sequences:

In [13]:
# pivot DMS data to get phenotypes
def pivot_for_ols_vars(df):
    ols_vars = (
        df
        .rename(
            columns={
                "phenotype": "full spike",
                "phenotype_RBD_only": "RBD",
                "phenotype_nonRBD_only": "non RBD",
            }
        )
        .assign(
            # group muts missing data from all phenotypes
            muts_from_DMS_clade_missing_data=lambda x: (
                x.groupby("clade")
                [f"muts_from_{dms_clade}_missing_data"]
                .transform(
                    lambda s: "; ".join(dict.fromkeys([m for ms in s.str.split("; ") for m in ms if m]))
                )
            ),
        )
        .rename(columns={f"muts_from_{dms_clade}": "muts_from_DMS_clade"})
        .pivot_table(
            index=[
                "clade",
                "R",
                "date",
                "muts_from_DMS_clade",
                "muts_from_DMS_clade_missing_data",
                "number sequences",
            ],
            columns="DMS_phenotype",
            values=["full spike", "RBD", "non RBD"],
        )
    )
    # flatten column names
    assert all(len(c) == 2 for c in ols_vars.columns.values)
    ols_vars.columns = [f"{pheno} ({domain})" for domain, pheno in ols_vars.columns.values]
    return ols_vars.reset_index()


def ols_unique_var_explained(var_endog, var_ols, vars, weight_var, full_r2):
    """Get unique variance explained by fitting model after removing each variable.

    https://blog.minitab.com/en/adventures-in-statistics-2/how-to-identify-the-most-important-predictor-variables-in-regression-models
    
    """
    unique_var = {}
    for vremove in vars:
        vremove_ols_model = statsmodels.api.WLS(
            endog=var_endog,
            exog=statsmodels.api.add_constant(ols_vars[[v for v in vars if v != vremove]]),
            weights=weights,
        )
        vremove_res_ols = vremove_ols_model.fit()
        unique_var[vremove] = full_r2 - vremove_res_ols.rsquared
    return unique_var

ols_vars = pivot_for_ols_vars(pango_dms_growth_df)

# https://www.einblick.ai/python-code-examples/ordinary-least-squares-regression-statsmodels/
for name, exog_vars, regression_chartfile, corr_chartfile in [
    (
        "full spike",
        [f"{c} (full spike)" for c in phenotypes],
        pango_dms_vs_growth_regression_html,
        pango_dms_vs_growth_corr_html
    ),
    (
        "separate RBD and non-RBD",
        [f"{c} ({d})" for d in ["RBD", "non RBD"] for c in phenotypes],
        pango_dms_vs_growth_regression_by_domain_html,
        pango_dms_vs_growth_corr_by_domain_html,
    ),
]:
    print(f"\n\nFitting for {name}:")
    weights = numpy.log(ols_vars["number sequences"])**2  # weight by log n sequences, so pass log**2
    ols_model = statsmodels.api.WLS(
        endog=ols_vars[["R"]],
        exog=statsmodels.api.add_constant(ols_vars[exog_vars]),
        weights=weights,
    )
    res_ols = ols_model.fit()
    unique_var = ols_unique_var_explained(ols_vars[["R"]], ols_vars, exog_vars, weights, full_r2=res_ols.rsquared)

    fitted_df = ols_vars.assign(DMS_predicted_growth=res_ols.predict())

    plot_size=160
    
    clade_selection = alt.selection_point(fields=["clade"], on="mouseover", empty=False)

    n_sequences_init = int(10 * math.log10(fitted_df["number sequences"].min())) / 10
    n_sequences_slider = alt.param(
        value=n_sequences_init,
        bind=alt.binding_range(
            name="minimum log10 number sequences in clade",
            min=n_sequences_init,
            max=math.log10(fitted_df["number sequences"].max() / 10),
        ),
    )

    # date slider: https://stackoverflow.com/a/67941109
    select_date = alt.selection_interval(encodings=["x"])
    date_slider = (
        alt.Chart(fitted_df[["clade", "date"]].drop_duplicates())
        .mark_bar(color="black")
        .encode(
            x=alt.X(
                "date",
                title="zoom bar to select clades by designation date",
                axis=alt.Axis(format="%b-%Y"),
            ),
            y=alt.Y("count()", title=["number", "clades"]),
        )
        .properties(width=1.5 * plot_size, height=45)
        .add_params(select_date)
    )
    
    base_growth_chart = (
        alt.Chart(fitted_df)
        .transform_filter(
            alt.expr.log(alt.datum["number sequences"]) / math.log(10) >= n_sequences_slider
        )
        .transform_filter(select_date)
        .encode(
            size=alt.Size(
                "number sequences",
                scale=alt.Scale(
                    type="log",
                    nice=False,
                    range=[15, 200],
                ),
                legend=alt.Legend(symbolStrokeWidth=0, symbolFillColor="gray"),
            ),
            strokeWidth=alt.condition(clade_selection, alt.value(2), alt.value(0.5)),
            strokeOpacity=alt.condition(clade_selection, alt.value(1), alt.value(0.5)),
            tooltip=[
                "clade",
                alt.Tooltip("R", title="growth rate", format=".1f"),
                alt.Tooltip("DMS_predicted_growth", title="DMS predicted growth", format=".1f"),
                alt.Tooltip("number sequences", format=".2g"),
                alt.Tooltip("date", title="designation date"),
                alt.Tooltip("muts_from_DMS_clade", title=f"muts from {dms_clade}"),
                alt.Tooltip("muts_from_DMS_clade_missing_data", title="muts missing DMS data"),
                *[alt.Tooltip(v, format=".2f") for v in exog_vars],  
            ],
        )
        .properties(width=plot_size, height=plot_size)
        .add_params(clade_selection, n_sequences_slider)
    )

    growth_charts = []
    simple_corr_charts = []
    for i, (dms_pheno, pheno) in enumerate(zip(
        exog_vars,
        itertools.cycle(phenotypes)
    )):
        assert dms_pheno.startswith(pheno)
        base_pheno_chart = (
            base_growth_chart
            .encode(
                y=alt.Y(
                    "R",
                    title="actual clade growth rate",
                    scale=alt.Scale(nice=False, padding=5, zero=False),
                    axis=None if i % len(phenotypes) else alt.Axis(),
                ),
            )
        )

        growth_charts.append(
            base_pheno_chart
            .encode(
                x=alt.X(
                    "DMS_predicted_growth",
                    title="DMS predicted clade growth",
                    scale=alt.Scale(nice=False, padding=5, zero=False),
                ),
                color=alt.Color(
                    dms_pheno,
                    title=None,
                    legend=alt.Legend(
                        orient="top",
                        titleFontSize=12,
                        gradientLength=plot_size,
                        gradientThickness=10,
                        offset=5,
                        tickCount=3,
                    ),
                    scale=alt.Scale(
                        range=polyclonal.plot.color_gradient_hex("lightgray", phenotype_colors[pheno], 40),
                        nice=False,
                    ),
                ),
            )
            .mark_circle(stroke="black", fillOpacity=0.6)
            .properties(
                title=alt.TitleParams(
                    text=dms_pheno,
                    subtitle=[
                        f"unique variance explained: {unique_var[dms_pheno] * 100:.1f}%",
                        f"coefficient: {res_ols.params[dms_pheno]:.1f} "
                        # https://stackoverflow.com/a/53966201
                        + f"\u00B1 {res_ols.bse[dms_pheno]:.1f}"
                    ],
                    subtitleFontSize=11,
                ),
            )
        )

        # get real and randomized P-value for simple correlations
        pheno_r, _ = scipy.stats.pearsonr(fitted_df["R"], fitted_df[dms_pheno])
        rand_rs = [
            scipy.stats.pearsonr(pivot_for_ols_vars(rand_df)["R"], pivot_for_ols_vars(rand_df)[dms_pheno])[0]
            for _, rand_df in pango_dms_growth_df_rand.groupby("randomize")
        ]
        rand_p = sum(pheno_r <= r for r in rand_rs) / len(rand_rs)
            
        simple_corr_charts.append(
            base_pheno_chart
            .transform_calculate(color_phenotype=f"'{pheno}'")
            .encode(
                x=alt.X(
                    dms_pheno,
                    scale=alt.Scale(nice=False, padding=5, zero=False),
                ),
                color=alt.Color(
                    "color_phenotype:N",
                    scale=alt.Scale(
                        range=list(phenotype_colors.values()),
                        domain=list(phenotype_colors.keys()),
                    ),
                    legend=None,
                ),
            )
            .mark_circle(stroke="black", fillOpacity=0.3, color=phenotype_colors[pheno])
            .properties(
                title=alt.TitleParams(
                    text=dms_pheno,
                    subtitle=f"Pearson r = {pheno_r:.2f} (P = {rand_p:.1g})",
                    subtitleFontSize=11,
                ),
            )
        )
            
    actual_r = math.sqrt(res_ols.rsquared)
    assert len(growth_charts) % len(phenotypes) == 0
    growth_chart = (
        alt.vconcat(
            alt.vconcat(
                *[
                    alt.hconcat(
                        *growth_charts[i * len(phenotypes): (i + 1) * len(phenotypes)], spacing=13
                    ).resolve_scale(color="independent")
                    for i in range(len(growth_charts) // len(phenotypes))
                ],
                spacing=13,
            ),
            date_slider,
        )
        .properties(
            title=alt.TitleParams(
                f"Weighted linear regression of DMS phenotypes vs clade growth (Pearson r = {actual_r:.2f})",
                subtitle=f"Showing all Pango clades descended from {', '.join(starting_clades)}",
                anchor="middle",
                fontSize=13,
                dy=-5,
            ),
        )
        .configure_axis(grid=False)
    )

    simple_corr_chart = (
        alt.vconcat(
            alt.vconcat(
                *[
                    alt.hconcat(
                        *simple_corr_charts[i * len(phenotypes): (i + 1) * len(phenotypes)], spacing=13
                    )
                    for i in range(len(simple_corr_charts) // len(phenotypes))
                ],
                spacing=13,
            ),
            date_slider,
        )
        .properties(
            title=alt.TitleParams(
                "Correlations of DMS phenotypes vs clade growth",
                subtitle=f"Showing all Pango clades descended from {', '.join(starting_clades)}",
                anchor="middle",
                fontSize=13,
                dy=-5,
            ),
        )
        .configure_axis(grid=False)
    )
    
    display(growth_chart)
    print(f"Saving to {regression_chartfile}")
    growth_chart.save(regression_chartfile)

    display(simple_corr_chart)
    print(f"Saving to {corr_chartfile}")
    simple_corr_chart.save(corr_chartfile)

    # fit randomized models and compute P-value for WLS based on R values
    print("Computing P-value for WLS randomizations")
    rand_r = []
    for randomseed, rand_df in pango_dms_growth_df_rand.groupby("randomize"):
        rand_ols_vars = pivot_for_ols_vars(rand_df)
        rand_ols_model = statsmodels.api.WLS(
            endog=rand_ols_vars[["R"]],
            exog=statsmodels.api.add_constant(rand_ols_vars[exog_vars]),
            # weight by log n sequences, so pass log**2
            weights=numpy.log(rand_ols_vars["number sequences"])**2,
        )
        rand_res_ols = rand_ols_model.fit()
        rand_r.append(math.sqrt(rand_res_ols.rsquared))
    n_rand_ge = sum(r >= actual_r for r in rand_r)
    pval = f"= {n_rand_ge / len(rand_r)}" if n_rand_ge else f"< {1 / len(rand_r)}"
    
    rand_r_hist = (
        alt.Chart(pd.DataFrame({"r": rand_r}))
        .encode(
            x=alt.X(
                "r",
                title="Pearson r",
                bin=alt.BinParams(step=0.02, extent=(0, 1)),
                scale=alt.Scale(domain=(0, 1)),
                axis=alt.Axis(values=[0, 0.2, 0.4, 0.6, 0.8, 1]),
            ),
            y=alt.Y("count()", title="number of randomizations"),
        )
        .mark_bar(color="black", opacity=0.65, align="right")
        .properties(width=250, height=130)
    )
    
    actual_r_line = (
        alt.Chart(pd.DataFrame({"r": [actual_r]}))
        .encode(x="r")
        .mark_rule(size=2, color="red", strokeDash=[4, 2])
    )
    
    pval_chart = (
        (rand_r_hist + actual_r_line)
        .configure_axis(grid=False)
        .properties(
            title=alt.TitleParams(
                f"P {pval}",
                subtitle=f"{n_rand_ge} of {len(rand_r)} randomizations 	\u2265 actual r of {actual_r:.2f}",
            ),
        )
    )
    
    display(pval_chart)



Fitting for full spike:


alt.VConcatChart(...)

Saving to results/compare_natural/pango_dms_vs_growth_regression.html


alt.VConcatChart(...)

Saving to results/compare_natural/pango_dms_vs_growth_corr.html
Computing P-value for WLS randomizations


...

alt.VConcatChart(...)

Saving to results/compare_natural/pango_dms_vs_growth_regression_by_domain.html


alt.VConcatChart(...)

Saving to results/compare_natural/pango_dms_vs_growth_corr_by_domain.html
Computing P-value for WLS randomizations


alt.LayerChart(...)

## Analyze all clades that differ from another clade by just one amino-acid mutation in spike
Get all clades that differ from their parent by just one spike mutation, also indicating if the spike mutation is only difference or there are also other non-spike differences:

In [14]:
def consolidate_reversions(r):
    """If there are reversions, combine with new mutations to get actual changes."""
    reverted = r["all_new_muts_reverted_from_ref"]
    new = r["all_new_muts_from_ref"]
    if not reverted:
        return new
    # get as dicts with key: val of (gene, site): (wt, mutant)
    reverted_dict = {
        (m.split(":")[0], m.split(":")[1][1: -1]): (m.split(":")[1][0], m.split(":")[1][-1])
        for m in reverted
    }
    new_dict = {
        (m.split(":")[0], m.split(":")[1][1: -1]): (m.split(":")[1][0], m.split(":")[1][-1])
        for m in new
    }
    muts = []
    for (gene, site), (rev_wt, rev_mutant) in reverted_dict.items():
        if (gene, site) in new_dict:
            new_wt, new_mutant = new_dict[(gene, site)]
            muts.append(f"{gene}:{rev_mutant}{site}{new_mutant}")
            del new_dict[(gene, site)]
        else:
            muts.append(f"{gene}:{rev_mutant}{site}{rev_wt}")
    for (gene, site), (new_wt, new_mutant) in new_dict.items():
        muts.append(f"{gene}:{new_wt}{site}{new_mutant}")
    return muts

pango_w_one_mut_from_parent = (
    pango_df
    .drop(columns=["n_child_clades", "muts_from_dms_clade"])
    .assign(
        all_new_muts=lambda x: x.apply(consolidate_reversions, axis=1),
        spike_new_muts=lambda x: x["all_new_muts"].map(lambda ms: [m.split(":")[1] for m in ms if m[0] == "S"]),
        n_new_spike_muts=lambda x: x["spike_new_muts"].map(len),
        n_new_all_muts=lambda x: x["all_new_muts"].map(len),
    )
    .query("n_new_spike_muts == 1")
    .assign(
        new_spike_mut=lambda x: x["spike_new_muts"].map(lambda ms: ms[0]),
        all_new_muts=lambda x: x["all_new_muts"].map(lambda ms: "; ".join(ms)),
        only_new_mut_is_spike=lambda x: x["n_new_spike_muts"] == x["n_new_all_muts"],
    )
    [["clade", "parent", "date", "new_spike_mut", "all_new_muts", "only_new_mut_is_spike"]]
    .reset_index(drop=True)
)

Add differences between clade and parent in DMS phenotype and growth rates:

In [15]:
pango_dms_growth_df

,clade,date,n_child_clades,muts_from_XBB.1.5,muts_from_XBB.1.5_missing_data,DMS_phenotype,phenotype,phenotype_RBD_only,phenotype_nonRBD_only,R,number sequences
0,XBB,2022-09-17,657,V252G; P486S,,ACE2 affinity,-0.60347,-0.57830,-0.02517,-0.936713,1650
1,XBB,2022-09-17,657,V252G; P486S,,cell entry,-0.21618,-0.24700,0.03082,-0.936713,1650
2,XBB,2022-09-17,657,V252G; P486S,,sera escape,-0.15530,-0.22690,0.07160,-0.936713,1650
3,XBB.1,2022-10-03,568,P486S,,ACE2 affinity,-0.57830,-0.57830,0.00000,1.204754,17510
4,XBB.1,2022-10-03,568,P486S,,cell entry,-0.24700,-0.24700,0.00000,1.204754,17510
...,...,...,...,...,...,...,...,...,...,...,...
907,EG.5.1.8,2023-09-11,0,Q52H; L455F; F456L,,cell entry,0.17176,0.09792,0.07384,76.763718,207
908,EG.5.1.8,2023-09-11,0,Q52H; L455F; F456L,,sera escape,0.27950,0.35170,-0.07220,76.763718,207
909,JG.3,2023-09-17,0,Q52H; L455F; F456L; S704L,,ACE2 affinity,-0.82190,-1.26070,0.43880,77.119926,215
910,JG.3,2023-09-17,0,Q52H; L455F; F456L; S704L,,cell entry,0.24621,0.09792,0.14829,77.119926,215


In [16]:
pango_dms_w_one_mut_from_parent = (
    pango_w_one_mut_from_parent
    .merge(
        (
            pango_dms_growth_df
            .rename(
                columns={f"muts_from_{dms_clade}_missing_data": "muts_missing_data"}
            )
            [["clade", "DMS_phenotype", "phenotype", "R", "number sequences", "muts_missing_data"]]
        ),
        on="clade",
        validate="one_to_many",
    )
    .merge(
        (
            pango_dms_growth_df
            .rename(
                columns={
                    "clade": "parent",
                    "phenotype": "parent_phenotype",
                    "R": "parent_R",
                    "number sequences": "parent number sequences",
                    f"muts_from_{dms_clade}_missing_data": "parent_muts_missing_data",
                }
            )
            [["parent", "DMS_phenotype", "parent_phenotype", "parent_R", "parent number sequences", "parent_muts_missing_data"]]
        ),
        on=["parent", "DMS_phenotype"],
        validate="many_to_one",
    )
    .assign(
        delta_phenotype=lambda x: x["phenotype"] - x["parent_phenotype"],
        delta_R=lambda x: x["R"] - x["parent_R"],
        site_missing_data=lambda x: x.apply(
            lambda r: set(
                m[1: -1]
                for m in r["muts_missing_data"].split(";") + r["parent_muts_missing_data"].split(";")
                if m
            ),
            axis=1,
        ),
        missing_data=lambda x: x.apply(
            lambda r: r["new_spike_mut"][1 : -1] in r["site_missing_data"],
            axis=1,
        ),
    )
    .query("not missing_data")
    .reset_index(drop=True)
    .drop(columns=["phenotype", "parent_phenotype", "R", "parent_R", "muts_missing_data", "parent_muts_missing_data", "site_missing_data", "missing_data"])
)

pango_dms_w_one_mut_from_parent.head()

,clade,parent,date,new_spike_mut,all_new_muts,only_new_mut_is_spike,DMS_phenotype,number sequences,parent number sequences,delta_phenotype,delta_R
0,XBB.1,XBB,2022-10-03,G252V,S:G252V,True,ACE2 affinity,17510,1650,0.02517,2.141467
1,XBB.2,XBB,2022-10-15,D253G,S:D253G,True,ACE2 affinity,1353,1650,-0.12340,1.143511
2,XBB.4,XBB,2022-10-20,K444R,ORF1a:G82D; S:K444R,False,ACE2 affinity,245,1650,0.04488,0.497720
3,XBB.1,XBB,2022-10-03,G252V,S:G252V,True,cell entry,17510,1650,-0.03082,2.141467
4,XBB.2,XBB,2022-10-15,D253G,S:D253G,True,cell entry,1353,1650,-0.12940,1.143511


In [23]:
pivoted_one_mut_from_parent = (
    pango_dms_w_one_mut_from_parent
    .query("new_spike_mut != 'L455F'")
#    .query("only_new_mut_is_spike")
    .pivot_table(
        index=["clade", "delta_R"],
        values="delta_phenotype",
        columns="DMS_phenotype",
    )
    .reset_index()
    .query(" and ".join(f"`{p}`.notnull()" for p in phenotypes))
)

ols_model = statsmodels.api.OLS(
    endog=pivoted_one_mut_from_parent[["delta_R"]],
    exog=statsmodels.api.add_constant(pivoted_one_mut_from_parent[phenotypes]),
)

res_ols = ols_model.fit()
display(res_ols.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                delta_R   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.262
Method:                 Least Squares   F-statistic:                     8.802
Date:                Wed, 11 Oct 2023   Prob (F-statistic):           5.87e-05
Time:                        15:50:59   Log-Likelihood:                -207.24
No. Observations:                  67   AIC:                             422.5
Df Residuals:                      63   BIC:                             431.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             6.0277      0.712      8.472      0.000       4.606       7.450
sera escape      26.5166      5.946      4.460      0.000      14.635      38.398
ACE2 affinity     6.2005      2.857      2.170      0.034       0.490      11.911
cell entry        1.0135      2.932      0.346      0.731      -4.846       6.873
==============================================================================
Omnibus:                        1.726   Durbin-Watson:                   2.083
Prob(Omnibus):                  0.422   Jarque-Bera (JB):                1.261
Skew:                           0.072   Prob(JB):                        0.532
Kurtosis:                       2.343   Cond. No.                         8.91
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
only_spike_muts = alt.param(
    value=False,
    bind=alt.binding_radio(
        options=[True, False],
        name="only show changes for clades with no non-spike differences",
    ),
)

delta_growth_dms_corr_base = (
    alt.Chart(pango_dms_w_one_mut_from_parent)
    .transform_calculate(keep_all=alt.expr.if_(only_spike_muts, False, True))
    .transform_filter(
        {
            "or": [
                alt.datum["only_new_mut_is_spike"],
                alt.datum["keep_all"],
            ]
        }
    )
    .transform_filter(
        (alt.expr.log(alt.datum["number sequences"]) / math.log(10) >= n_sequences_slider)
        and (alt.expr.log(alt.datum["parent number sequences"]) / math.log(10) >= n_sequences_slider)
    )
    .transform_filter(select_date)
    .add_params(clade_selection, n_sequences_slider, only_spike_muts)
    .properties(width=plot_size, height=plot_size)
)

delta_growth_dms_corr_scatter = (
    delta_growth_dms_corr_base
    .encode(
        alt.X(
            "delta_phenotype",
            title="change in DMS phenotype",
            scale=alt.Scale(nice=False, padding=3),
        ),
        alt.Y(
            "delta_R",
            title="change in growth rate",
            scale=alt.Scale(nice=False, padding=3),
        ),
        color=alt.Color(
            "DMS_phenotype",
            sort=phenotypes,
            scale=alt.Scale(range=[phenotype_colors[p] for p in phenotypes]),
            legend=None,
        ),
        strokeWidth=alt.condition(clade_selection, alt.value(2), alt.value(0)),
        opacity=alt.condition(clade_selection, alt.value(1), alt.value(0.3)),
        size=alt.condition(clade_selection, alt.value(80), alt.value(50)),
        tooltip=[
            "clade",
            "parent",
            alt.Tooltip("delta_R", title="change in growth", format=".1f"),
            "DMS_phenotype",
            alt.Tooltip("delta_phenotype", title="change in DMS phenotype", format=".2f"),
            "new_spike_mut",
            "all_new_muts",
            "date",
            "number sequences",
            "parent number sequences",
        ],
    )
    .mark_circle(size=40, stroke="black", strokeOpacity=1)
)

delta_growth_dms_corr_r = (
    delta_growth_dms_corr_base
    .transform_regression("delta_phenotype", "delta_R", params=True)
    .transform_calculate(
        r=alt.expr.if_(
            alt.datum["coef"][1] >= 0,
            alt.expr.sqrt(alt.datum["rSquared"]),
            -alt.expr.sqrt(alt.datum["rSquared"]),
        ),
        label='"r = " + format(datum.r, ".2f")',
    )
    .mark_text(align="left", color="black", fontWeight=500, fontSize=12)
    .encode(
        x=alt.value(3),
        y=alt.value(8),
        text=alt.Text("label:N"),
    )
)

delta_growth_dms_corr = (
    alt.vconcat(
        (
            (delta_growth_dms_corr_scatter + delta_growth_dms_corr_r)
            .facet(
                alt.Column(
                    "DMS_phenotype",
                    header=alt.Header(
                        title=None,
                        labelFontSize=13,
                        labelFontStyle="bold",
                        labelPadding=3,
                    ),
                    sort=phenotypes,
                ),
                spacing=4,
            )
            .resolve_scale(x="independent")
        ),
        date_slider,
    )
    .properties(
        title=alt.TitleParams(
            "Change in DMS phenotype vs change in growth for clades differing by one spike mutation",
            subtitle=f"Showing all Pango clades descended from {', '.join(starting_clades)}",
            anchor="middle",
            fontSize=13,
            dy=-5,
        ),
    )
    .configure_axis(grid=False)
)

delta_growth_dms_corr

alt.VConcatChart(...)

## Distributions of DMS mutation effects in clades with growth estimates versus all mutations
These are mutations relative to the DMS strain.

In [ ]:
muts_in_clades = collections.Counter(
    pango_dms_growth_df
    [f"muts_from_{dms_clade}"]
    .pipe(lambda s: s[s != ""])
    .str.split("; ")
    .explode()
)
print(f"There are {len(muts_in_clades)} mutations found in clades with growth estimates")

all_muts_dms = (
    dms_summary
    .query("wildtype != mutant")
    .assign(mutation=lambda x: x["wildtype"] + x["site"].astype(str) + x["mutant"])
    .assign(region=lambda x: x["region"].where(x["region"] == "RBD", "non RBD"))
    .melt(
        id_vars=["mutation", "region"],
        value_vars=phenotypes,
        var_name="DMS_phenotype",
        value_name="phenotype",
    )
    .query("phenotype.notnull()") 
)

all_muts_dms = pd.concat(
    [
        all_muts_dms.assign(mutation_type="any", count=1),
        all_muts_dms.query("mutation in @muts_in_clades").assign(
            mutation_type="in Pango clade",
            count=lambda x: x["mutation"].map(muts_in_clades),
        ),
    ]
)

for pheno in phenotypes:
    
    base_hist = (
        alt.Chart(
            all_muts_dms
            .query("DMS_phenotype == @pheno")
            .drop(columns=["DMS_phenotype", "mutation"])
        )
        .encode(
            x=alt.X("phenotype", bin=alt.BinParams(maxbins=50)),
            y=alt.Y("sum(count)", title="mutations"),
            color=alt.value(phenotype_colors[pheno]),
            row=alt.Row("mutation_type", title=None, spacing=5),
        )
        .properties(width=200, height=75, title=pheno)
        .mark_bar()
        .resolve_scale(y="independent")
    )
    display(base_hist)